In [1]:
def get_aisis_data(department, subject='ALL'):
    from aisis_getin import data # for log in data
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import Select, WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    import pandas as pd
    import time
    PATH ='C:\Program Files (x86)\chromedriver.exe'
    driver = webdriver.Chrome(PATH)
    
    #logging in 
    login_url = 'https://aisis-z2.ateneo.edu/j_aisis/displayLogin.do'
    driver.get(login_url)
    print(driver.title)
    login_username = driver.find_element_by_name("userName")
    login_password = driver.find_element_by_name("password")
    #inputting login contents
    login_username.send_keys(data['userName'])
    login_password.send_keys(data['password'])
    login_username.send_keys(Keys.RETURN)
    driver.implicitly_wait(4)
    #go to class schedules
    driver.find_element_by_link_text("CLASS SCHEDULE").click()
    #display schedule of classes
    driver.implicitly_wait(2)
    #select department
    department_select = Select(driver.find_element_by_name("deptCode"))
    department_select.select_by_visible_text(department)#to be chosen
    #select subject
    subject_select = Select(driver.find_element_by_name("subjCode"))
    subject_select.select_by_visible_text(subject)
    #click Display Class Schedule
    display_button_xpath = '/html/body/div/table/tbody/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[4]/td/table/tbody/tr[3]/td/table/tbody/tr/td/form/table/tbody/tr/td/table/tbody/tr[4]/td/input'
    driver.find_element_by_xpath(display_button_xpath).click()
    #getting schedule contents
    driver.implicitly_wait(2)
    tbl = driver.find_element_by_xpath("/html/body/div/table/tbody/tr[2]").get_attribute('outerHTML')
    schedule_info  = pd.read_html(tbl,header = 0)
    df = schedule_info[8]
    driver.quit()
    return df

In [2]:
ptp_data ={}
def collect_ptp_data(prof_full_name): #Last, First MI.
    from secrets import data # for log in data
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import Select, WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.common.action_chains import ActionChains
    import pandas as pd
    import time
    import requests
    from bs4 import BeautifulSoup
    
    PATH ='C:\Program Files (x86)\chromedriver.exe'
    option = Options()

    #turn off chrome interruptions
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")

    # Pass the argument 1 to allow and 2 to block
    option.add_experimental_option("prefs", { 
        "profile.default_content_setting_values.notifications": 1 
    })

    driver = webdriver.Chrome(chrome_options=option, executable_path=PATH)
    actions = ActionChains(driver)

    #logging in |
    my_email = data['email']
    my_password = data['pass']

    login_url = 'https://www.facebook.com/login'
    driver.get(login_url)
    print(driver.title)
    login_username = driver.find_element_by_id("email")
    login_password = driver.find_element_by_id("pass")

    #inputting login contents
    login_username.send_keys(my_email)
    login_password.send_keys(my_password)
    login_username.send_keys(Keys.RETURN)


    #going to profs to pick
    #make profs to pick first shortcut for facebook account
    ptp_xpath ='//*[@id="mount_0_0"]/div/div[1]/div[1]/div[3]/div/div/div[1]/div[1]/div/div[1]/div/div/div[1]/div/div/div[1]/div[2]/ul/li[1]/div/a/div[1]/div[2]/div/div/div/div/span'
    driver.implicitly_wait(5)
    driver.find_element_by_xpath(ptp_xpath).click()
    driver.implicitly_wait(3)

    #change to mobile mode
    driver.get('https://m.facebook.com/groups/1568550996761154')
    driver.implicitly_wait(3)

    #in profs to pick 
    #click search
    start_search_xpath = '//*[@id="MRoot"]/div/div[2]/div/div[1]/div/div[3]/a'
    driver.implicitly_wait(5)
    driver.find_element_by_xpath(start_search_xpath).click()

    #search prof
    driver.implicitly_wait(3)
    type_xpath = "//*[@id='main-search-input']"
    search_prof = driver.find_element_by_xpath(type_xpath)
    driver.implicitly_wait(3)
    search_prof.send_keys(prof_full_name)
    search_prof.send_keys(Keys.RETURN)
    prof_dict = {}

    #switch to mbasic
    currentURL = driver.current_url
    newURL = currentURL[:9] + 'basic' + currentURL[9:]
    driver.get(newURL)
    driver.implicitly_wait(3)

    #collect post links
    see_more = driver.find_element_by_xpath("//span[text()='See More Results']")
    actions.move_to_element(see_more).click()
    find_posts = driver.find_elements_by_link_text("Full Story")#changes made
    posts = [post.get_attribute('href') for post in find_posts]

    for post in posts:
        #go to post link
        driver.get(post)
        driver.implicitly_wait(3)

        #get post
        main_post = driver.find_element_by_class_name("bu").text

        #switch back to m.facebook
        post_url = driver.current_url
        post_newURL = post_url[:9]+post_url[14:]
        driver.get(post_newURL)

        #get comments
        comments_list = []
        html = driver.page_source
        page_soup =BeautifulSoup(html, "html.parser")
        html_comment = page_soup.findAll("div",{'data-sigil':"comment-body"})
        for comment in html_comment:
            comments_list.append(comment.text)

        #add to post dictionary
        prof_dict[main_post] = comments_list 

    #add to dictionary for all profs    
    ptp_data[prof_full_name]=prof_dict

    driver.quit()
    print('done')

In [3]:
data_frame = get_aisis_data(department = 'MATHEMATICS')
prof_list = data_frame["Instructor"].tolist()
prof_list = set(prof_list)
prof_list.remove("TO BE ARRANGED")
print(len(prof_list))
count = 0
for prof in prof_list:
    print(f"doing {prof}")
    collect_ptp_data(prof)
    count+=1
    print(count)
    
print("ALL DONE")

AISIS Online :: Ateneo Integrated Student Information System
33
doing SIA, Calvin


C:\Users\joshu\anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: use options instead of chrome_options


Log into Facebook | Facebook
done
doing GARCES, IAN JUNE L.
Log into Facebook | Facebook
done
doing BUOT, JUDE C.
Log into Facebook | Facebook
done
doing SANTOS, DURWIN C.
Log into Facebook | Facebook
done
doing SEGISMUNDO, Jasper John
Log into Facebook | Facebook
done
doing EDEN, RICHARD B.
Log into Facebook | Facebook
done
doing MARCELO, REGINALDO M.
Log into Facebook | Facebook
done
doing MIRO, EDEN DELIGHT P.
Log into Facebook | Facebook
done
doing DUMBRIQUE, JAKOV IVAN S.
Log into Facebook | Facebook
done
doing TOLENTINO, MARK ANTHONY C.
Log into Facebook | Facebook
done
doing TABARES, WINFER C.
Log into Facebook | Facebook
done
doing VASWANI, SERENA M.
Log into Facebook | Facebook
done
doing ONG, LU CHRISTIAN S.
Log into Facebook | Facebook
done
doing GO, CLARK KENDRICK C.
Log into Facebook | Facebook
done
doing BATALLER, RAMIL T.
Log into Facebook | Facebook
done
doing LEE-CHUA, QUEENA N.
Log into Facebook | Facebook
done
doing GUZON, ANGELA FATIMA H.
Log into Facebook | Faceboo

In [35]:
import json

with open('ptp_scraped_dict.json', 'w', encoding = 'utf-8', newline="") as file: 
    json.dumps(ptp_data)